In [2]:
# !pip uninstall umap --yes
# !pip uninstall umap-learn --yes
# !pip3 install umap-learn
# !conda uninstall umap-learn
# !pip freeze
# !conda install -y -c conda-forge umap-learn --force-reinstall

In [3]:
import glob
from pathlib import Path
from umap import UMAP
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy.stats
from kymatio import Scattering2D
import json
%matplotlib notebook

# base_dir = '/n/data1/hms/neurobio/sabatini/josh/analysis/ROI_net_training/20220510_testingSimCLRscript/jobNum_*/'
base_dir = '/n/data1/hms/neurobio/sabatini/josh/analysis/ROI_net_training/simCLR_paramsweep_temp'
jns = sorted(glob.glob(str(Path(base_dir) / 'jobNum_*')))

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
pth_fn_lst = []
log_fn_lst = []
loss_fn_lst = []
params_fn_lst = []
run_outputs_fn_lst = []
sbatch_config_fn_lst = []
base_py_fn_lst = []

for jn in jns:
    print(jn)
    
    pth_fn = glob.glob(str(Path(jn) / '*.pth'))
    log_fn = glob.glob(str(Path(jn) / 'log.txt'))
    loss_fn = glob.glob(str(Path(jn) / 'loss.npy'))
    params_fn = glob.glob(str(Path(jn) / 'params.json'))
    run_outputs_fn = glob.glob(str(Path(jn) / 'run_outputs.json'))
    sbatch_config_fn = glob.glob(str(Path(jn) / 'sbatch_config.sh'))
    base_py_fn = glob.glob(str(Path(jn) / '*.py'))
    
    assert len(pth_fn) == 1
    assert len(log_fn) == 1
    assert len(loss_fn) == 1
    assert len(params_fn) == 1
    assert len(run_outputs_fn) == 1
    assert len(sbatch_config_fn) == 1
    assert len(base_py_fn) == 1
    
    pth_fn_lst += pth_fn
    log_fn_lst += log_fn
    loss_fn_lst += loss_fn
    params_fn_lst += params_fn
    run_outputs_fn_lst += run_outputs_fn
    sbatch_config_fn_lst += sbatch_config_fn
    base_py_fn_lst += base_py_fn
    
    break


with open(params_fn[0], 'rb') as f:
    params = json.load(f)

/n/data1/hms/neurobio/sabatini/josh/analysis/ROI_net_training/simCLR_paramsweep_temp/jobNum_0


In [9]:
import torchvision
def model_from_params(params, pref_log_all_steps=False):
    base_model_frozen = torchvision.models.__dict__[params['torchvision_model']](pretrained=True)
    for param in base_model_frozen.parameters():
        param.requires_grad = False
    if pref_log_all_steps:
        write_to_log(path_log=path_saveLog, text=f'time:{time.ctime()}  imported pretrained model')

    ### Make combined model

    ## Tacking on the latent layers needs to be done in a few steps.

    ## 0. Chop the base model
    ## 1. Tack on a pooling layer to reduce the size of the convlutional parameters
    ## 2. Determine the size of the output (internally done in ModelTackOn)
    ## 3. Tack on a linear layer of the correct size  (internally done in ModelTackOn)

    if pref_log_all_steps:
        write_to_log(path_log=path_saveLog, text=f'time:{time.ctime()}  making combined model...')

    model_chopped = torch.nn.Sequential(list(base_model_frozen.children())[0][:params['n_block_toInclude']])  ## 0.
    model_chopped_pooled = torch.nn.Sequential(model_chopped, torch.nn.AdaptiveAvgPool2d(output_size=1), torch.nn.Flatten())  ## 1.

    image_out_size = list(dataset_train[0][0][0].shape)
    data_dim = tuple([1] + list(image_out_size))
    
    model = ModelTackOn(
    #     model_chopped.to('cpu'),
        model_chopped_pooled.to('cpu'),
        base_model_frozen.to('cpu'),
        data_dim=data_dim,
        pre_head_fc_sizes=params['pre_head_fc_sizes'], 
        post_head_fc_sizes=params['post_head_fc_sizes'], 
        classifier_fc_sizes=None,
        nonlinearity=params['head_nonlinearity'],
        kwargs_nonlinearity={},
    )
    return base_model_frozen, model_chopped, model_chopped_pooled, image_out_size, data_dim, model

model_from_params(params, pref_log_all_steps=False)

KeyError: 'convnext_tiny'

In [10]:
torchvision.models.__dict__

{'__name__': 'torchvision.models',
 '__doc__': None,
 '__package__': 'torchvision.models',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7f9a57cc62e0>,
 '__spec__': ModuleSpec(name='torchvision.models', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f9a57cc62e0>, origin='/home/joz608/.conda/envs/jupyter_launcher/lib/python3.8/site-packages/torchvision/models/__init__.py', submodule_search_locations=['/home/joz608/.conda/envs/jupyter_launcher/lib/python3.8/site-packages/torchvision/models']),
 '__path__': ['/home/joz608/.conda/envs/jupyter_launcher/lib/python3.8/site-packages/torchvision/models'],
 '__file__': '/home/joz608/.conda/envs/jupyter_launcher/lib/python3.8/site-packages/torchvision/models/__init__.py',
 '__cached__': '/home/joz608/.conda/envs/jupyter_launcher/lib/python3.8/site-packages/torchvision/models/__pycache__/__init__.cpython-38.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other o

In [12]:
!conda install -c conda-forge torchvision==0.12

failed

CondaError: KeyboardInterrupt



In [38]:
# final_code = ''

# trgrd = False
# with open(base_py[0], 'r') as f:
# #     display(f.readlines())
#     for line in f.readlines():
#         if 'Import pretrained model' in line and 'break':
#             break
#         if 'class ModelTackOn' in line:
#             trgrd = True
#         if trgrd:
#             final_code += line
# #             print(line, end='')
base_py

['/n/data1/hms/neurobio/sabatini/josh/analysis/ROI_net_training/simCLR_paramsweep_temp/jobNum_0/train_ROInet_simCLR_20220508.py']

In [41]:
import torchvision

base_model_frozen = torchvision.models.__dict__[params['torchvision_model']](pretrained=True)

for param in base_model_frozen.parameters():
    param.requires_grad = False

if pref_log_all_steps:
    write_to_log(path_log=path_saveLog, text=f'time:{time.ctime()}  imported pretrained model')



### Make combined model

## Tacking on the latent layers needs to be done in a few steps.

## 0. Chop the base model
## 1. Tack on a pooling layer to reduce the size of the convlutional parameters
## 2. Determine the size of the output (internally done in ModelTackOn)
## 3. Tack on a linear layer of the correct size  (internally done in ModelTackOn)

if pref_log_all_steps:
    write_to_log(path_log=path_saveLog, text=f'time:{time.ctime()}  making combined model...')

model_chopped = torch.nn.Sequential(list(base_model_frozen.children())[0][:params['n_block_toInclude']])  ## 0.
model_chopped_pooled = torch.nn.Sequential(model_chopped, torch.nn.AdaptiveAvgPool2d(output_size=1), torch.nn.Flatten())  ## 1.

image_out_size = list(dataset_train[0][0][0].shape)
data_dim = tuple([1] + list(image_out_size))

## 2. , 3.
model = ModelTackOn(
#     model_chopped.to('cpu'),
    model_chopped_pooled.to('cpu'),
    base_model_frozen.to('cpu'),
    data_dim=data_dim,
    pre_head_fc_sizes=params['pre_head_fc_sizes'], 
    post_head_fc_sizes=params['post_head_fc_sizes'], 
    classifier_fc_sizes=None,
    nonlinearity=params['head_nonlinearity'],
    kwargs_nonlinearity={},
)

TypeError: list indices must be integers or slices, not str

In [39]:
import importlib.util
spec = importlib.util.spec_from_file_location(base_py[0].split('/')[-1][:-3], base_py[0])
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
module.ModelTackOn()

model = module.ModelTackOn(
    #     model_chopped.to('cpu'),
        model_chopped_pooled.to('cpu'),
        base_model_frozen.to('cpu'),
        data_dim=data_dim,
        pre_head_fc_sizes=params['pre_head_fc_sizes'], 
        post_head_fc_sizes=params['post_head_fc_sizes'], 
        classifier_fc_sizes=None,
        nonlinearity=params['head_nonlinearity'],
        kwargs_nonlinearity={},
    )

sys.version_info(major=3, minor=8, micro=12, releaselevel='final', serial=0)
jupyter_launcher


TypeError: __init__() missing 2 required positional arguments: 'base_model' and 'un_modified_model'

In [ ]:
labels_sesh1 = np.load(r'/media/rich/bigSSD/for_Josh/SimCLR-Label-Data/mouse2_6__20210409/labels_round2_sesh1.npy')
labels_sesh2 = np.load(r'/media/rich/bigSSD/for_Josh/SimCLR-Label-Data/mouse2_6__20210409/labels_round2_sesh2.npy')

In [ ]:
def helper_make_dataset(X):
    out = dataset.dataset_simCLR(
        X=torch.as_tensor(X, device='cpu', dtype=torch.float32),
        y=torch.as_tensor(torch.zeros(X.shape[0]), device='cpu', dtype=torch.float32),
        n_transforms=1,
        class_weights=np.array([1]),
        transform=scripted_transforms_classifier,
        DEVICE='cpu',
        dtype_X=torch.float32,
    )
    return out

def helper_make_dataloader(ds):
    out = torch.utils.data.DataLoader( 
        ds,
        batch_size=128,
        shuffle=False,
        drop_last=False,
#         pin_memory=True,
#         num_workers=36,
#         persistent_workers=True,
#         prefetch_factor=2
    )
    return out

In [ ]:
dataset_labeled_clean = helper_make_dataset(images_labeled_clean)
dataloader_labeled_clean = helper_make_dataloader(dataset_labeled_clean)

In [ ]:
DEVICE = torch_helpers.set_device(use_GPU=True)

In [ ]:
model.eval()
model.to(DEVICE)
features_nn = torch.cat([model.base_model(data[0][0].to(DEVICE)).detach().cpu() for data in dataloader_labeled_clean], dim=0)

In [ ]:
def get_balanced_sample_weights(labels):
    labels = np.int64(labels.copy())
    counts, vals = np.histogram(labels, bins=np.concatenate((np.unique(labels), [labels.max()+1])))
    vals = vals[:-1]

    n_labels = len(labels)
    weights = n_labels / counts
    
    sample_weights = np.array([weights[l] for l in labels])
    
    return sample_weights

In [ ]:
def get_latents_swt(sfs, swt, device_model):
    sfs = torch.as_tensor(np.ascontiguousarray(sfs[None,...]), device=device_model, dtype=torch.float32)
    latents_swt = swt(sfs[None,...]).squeeze()
    latents_swt = latents_swt.reshape(latents_swt.shape[0], -1)
    return latents_swt


device_model = torch_helpers.set_device(use_GPU=True)

scattering = Scattering2D(J=2, L=8, shape=images_labeled_clean[0].shape[-2:])
if device_model != 'cpu':
    scattering = scattering.cuda()

latents_swt = get_latents_swt(images_labeled_clean, scattering.cuda(), device_model).cpu()

In [ ]:
torch.corrcoef(features_nn.T).shape

In [ ]:
plt.figure()
plt.imshow(features_nn, aspect='auto',)

In [ ]:
plt.figure()
plt.imshow(scipy.stats.zscore(features_nn, axis=0), aspect='auto', vmin=-0.8, vmax=1,)

In [ ]:
features_nn_z = scipy.stats.zscore(features_nn.numpy(), axis=0)
features_nn_z = features_nn_z[:, ~np.isnan(features_nn_z[0,:])]
features_nn_z = torch.as_tensor(features_nn_z, dtype=torch.float32)

In [ ]:
features_nn_z.shape

In [ ]:
comp_nn, scores_nn, SVs, EVR_nn = decomposition.torch_pca(features_nn_z, rank=40, zscore=True)
comp, scores_swt, SVs, EVR_swt = decomposition.torch_pca(latents_swt, rank=20)
comp, scores_image, SVs, EVR_image = decomposition.torch_pca(images_labeled_clean.reshape(images_labeled_clean.shape[0], -1), rank=5)

In [ ]:
plt.figure()
plt.plot(EVR_nn)
plt.yscale('log')

In [ ]:
features_nn.shape

In [ ]:
plt.figure()
plt.imshow(torch.corrcoef(features_nn.T))

In [ ]:
features_norm = torch.cat([val / torch.std(val, dim=0).mean() for val in [scores_nn, scores_swt]], dim=1)

In [ ]:
features_train, features_val, labels_train, labels_val = sklearn.model_selection.train_test_split(features_norm, labels_clean, test_size=0.3)
print(f'train / test shapes: images_labeled_train, images_labeled_val, labels_train, labels_val: {features_train.shape, features_val.shape, labels_train.shape, labels_val.shape}')

In [ ]:
acc_train, acc_val = [], []
C_toUse = np.array([1000, 100, 10,1,0.1,0.01,0.001])
# C_toUse = np.array([10**-4, 10**-5, 10**-6, 10**-7])
# C_toUse = np.array([10**2, 10**3, 10**4, 10**5])
for C in C_toUse:
    logreg = sklearn.linear_model.LogisticRegression(
        solver='lbfgs',
#         solve='sag'
#         solve='saga'
#         solver='newton-cg',
#         solver='liblinear',
        max_iter=6000, 
        C=C,
        fit_intercept=True, 
        class_weight='balanced',
    #     n_jobs=-1
    )
    logreg.fit(features_train, labels_train)

    acc = logreg.score(features_train, labels_train, sample_weight=get_balanced_sample_weights(labels_train))
    acc_train.append(acc)

    acc = logreg.score(features_val, labels_val, sample_weight=get_balanced_sample_weights(labels_val))
    acc_val.append(acc)
    
    

In [ ]:
plt.figure()
plt.plot(C_toUse, acc_train)
plt.plot(C_toUse, acc_val)
plt.xscale('log')
plt.xlabel('C')
plt.ylabel('acc')
plt.legend(['train', 'test']);

print(f'best val score: {max(acc_val)}')
print(f'best C value: {C_toUse[np.argmax(acc_val)]}')

In [ ]:
logreg = sklearn.linear_model.LogisticRegression(
        solver='lbfgs',
#         solver='sag',
#         solver='saga',
#         solver='newton-cg',
#         solver='liblinear',
    max_iter=8000, 
    C=10**(0),
    fit_intercept=True, 
    class_weight='balanced',
#     n_jobs=-1
).fit(features_train, labels_train)

In [ ]:
svm = sklearn.svm.SVC(
    C=10**(1), 
    kernel='rbf',
    degree=3, 
    gamma='scale',
    coef0=0.0, 
    shrinking=True,
    probability=False, 
    tol=0.001, 
    cache_size=200,
    class_weight='balanced',
    verbose=False, 
    max_iter=- 1, 
    decision_function_shape='ovr',
    break_ties=False,
    random_state=None
)
svm.fit(features_train, labels_train)

In [ ]:
proba = logreg.predict_proba(features_train)
preds = np.argmax(proba, axis=1)

# preds = svm.predict(features_train_norm).astype(np.int32)

cm = classification.confusion_matrix(preds, labels_train.astype(np.int32))

plt.figure()
sns.heatmap(cm, annot=True, annot_kws={"size": 16}, vmax=1., cmap=plt.get_cmap('gray'))
plt.title('train');

proba = logreg.predict_proba(features_val)
preds = np.argmax(proba, axis=1)

cm = classification.confusion_matrix(preds, labels_val.astype(np.int32))

plt.figure()
sns.heatmap(cm, annot=True, annot_kws={"size": 16}, vmax=1., cmap=plt.get_cmap('gray'))
plt.title('val');

In [ ]:
labels_sesh12cat = np.concatenate((labels_sesh1, labels_sesh2), axis=0)
labels_sesh21cat = np.concatenate((labels_sesh2, labels_sesh1), axis=0)
plt.figure()
sns.heatmap(
    classification.confusion_matrix(labels_sesh12cat.astype(np.int32), labels_sesh21cat.astype(np.int32)),
    annot=True, 
    annot_kws={"size": 16}, 
    vmax=1., 
    cmap=plt.get_cmap('gray')
)
plt.title('relabeling')

In [ ]:
[(name, params.shape) for name, params in model.state_dict().items()]

In [ ]:
layer_1 = model.state_dict()['base_model.0.0.0.weight'].cpu()
# layer_2 = model.state_dict()['base_model.0.6.3.block.1.0.weight'].cpu()
# layer_3 = model.state_dict()['base_model.0.7.0.block.1.0.weight'].cpu()
# layer_4 = model.state_dict()['base_model.0.7.0.block.1.0.weight'].cpu()


plotting_helpers.plot_image_grid(torch.cat([arr for arr in layer_1], dim=0), grid_shape=(15,15), kwargs_imshow={'vmax': 0.2});
# plotting_helpers.plot_image_grid(torch.cat([arr for arr in layer_2], dim=0), grid_shape=(6,6), kwargs_imshow={'vmax': 0.2});
# plotting_helpers.plot_image_grid(torch.cat([arr for arr in layer_3], dim=0), grid_shape=(4,6), kwargs_imshow={'vmax': 0.2});

In [ ]:
umap = UMAP(
    n_neighbors=30,
    n_components=2,
    metric='euclidean',
    metric_kwds=None,
    output_metric='euclidean',
    output_metric_kwds=None,
    n_epochs=None,
    learning_rate=1.0,
    init='spectral',
    min_dist=0.1,
    spread=1.0,
    low_memory=True,
    n_jobs=-1,
    set_op_mix_ratio=1.0,
    local_connectivity=1.0,
    repulsion_strength=1.0,
    negative_sample_rate=5,
    transform_queue_size=4.0,
    a=None,
    b=None,
    random_state=None,
    angular_rp_forest=False,
    target_n_neighbors=-1,
    target_metric='categorical',
    target_metric_kwds=None,
    target_weight=0.5,
    transform_seed=42,
    transform_mode='embedding',
    force_approximation_algorithm=False,
    verbose=False,
    tqdm_kwds=None,
    unique=False,
    densmap=False,
    dens_lambda=2.0,
    dens_frac=0.3,
    dens_var_shift=0.1,
    output_dens=False,
    disconnection_distance=None,
    precomputed_knn=(None, None, None),
)

In [ ]:
embeddings = umap.fit_transform(features_nn)

In [ ]:
plt.figure()
plt.scatter(embeddings[:,0], embeddings[:,1], s=5, c=labels_clean, cmap='gist_rainbow')

In [ ]:
plt.figure()
plt.scatter(scores_nn[:,1], scores_nn[:,0], c=labels_clean)